Creating Spark Session Here

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ipl Bolte").getOrCreate()
# spark

In [14]:
file_path_deliveries = './deliveries.csv'
df_del = spark.read.format('csv').option("header","true").load(file_path_deliveries)

In [16]:
df_del.show()

+--------+------+--------------------+--------------------+----+----+-----------+---------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|match_id|inning|        batting_team|        bowling_team|over|ball|     batter|   bowler|non_striker|batsman_runs|extra_runs|total_runs|extras_type|is_wicket|player_dismissed|dismissal_kind|fielder|
+--------+------+--------------------+--------------------+----+----+-----------+---------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   1| SC Ganguly|  P Kumar|BB McCullum|           0|         1|         1|    legbyes|        0|              NA|            NA|     NA|
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   2|BB McCullum|  P Kumar| SC Ganguly|           0|         0|         0|       NULL|        0|              NA|            NA|    

In [18]:
file_path_matches = './matches.csv'
df_match = spark.read.format('csv').option("header","true").load(file_path_matches)
df_match.show()

+------+------+----------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+-----------+------------+----------+-----------+--------------+
|    id|season|match_type|player_of_match|               venue|               team1|               team2|         toss_winner|toss_decision|              winner| result|result_margin|target_runs|target_overs|super_over|    umpire1|       umpire2|
+------+------+----------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+-----------+------------+----------+-----------+--------------+
|335982|  2008|    League|    BB McCullum|M Chinnaswamy Sta...|Royal Challengers...|Kolkata Knight Ri...|Royal Challengers...|        field|Kolkata Knight Ri...|   runs|          140|        223|          20|         N|  Asad Rauf|   RE Koertzen|
|335983|  20

In [20]:
df_match.columns

['id',
 'season',
 'match_type',
 'player_of_match',
 'venue',
 'team1',
 'team2',
 'toss_winner',
 'toss_decision',
 'winner',
 'result',
 'result_margin',
 'target_runs',
 'target_overs',
 'super_over',
 'umpire1',
 'umpire2']

In [28]:
df_match.select('winner').distinct().show()

+--------------------+
|              winner|
+--------------------+
| Sunrisers Hyderabad|
|Lucknow Super Giants|
| Chennai Super Kings|
|      Gujarat Titans|
|                  NA|
|Rising Pune Super...|
|     Deccan Chargers|
|Kochi Tuskers Kerala|
|    Rajasthan Royals|
|       Gujarat Lions|
|Royal Challengers...|
|Kolkata Knight Ri...|
|        Punjab Kings|
|       Pune Warriors|
|      Delhi Capitals|
|      Mumbai Indians|
+--------------------+



NOW Clearing the matches where there is no winner and then making our data set clean


In [37]:
df_match[df_match['winner']=='NA'].show()

+-------+------+----------+---------------+--------------------+--------------------+-------------------+--------------------+-------------+------+---------+-------------+-----------+------------+----------+---------------+-------------+
|     id|season|match_type|player_of_match|               venue|               team1|              team2|         toss_winner|toss_decision|winner|   result|result_margin|target_runs|target_overs|super_over|        umpire1|      umpire2|
+-------+------+----------+---------------+--------------------+--------------------+-------------------+--------------------+-------------+------+---------+-------------+-----------+------------+----------+---------------+-------------+
| 501265|  2011|    League|             NA|    Feroz Shah Kotla|      Delhi Capitals|      Pune Warriors|      Delhi Capitals|          bat|    NA|no result|         NULL|       NULL|        NULL|         N|      SS Hazare|    RJ Tucker|
| 829763|  2015|    League|             NA|M Chi

AttributeError: 'NoneType' object has no attribute 'style'

In [56]:
df_match_cleaned = df_match.where(df_match["winner"] != "NA")

In [58]:
df_match_cleaned[df_match_cleaned['winner']=="NA"].show()

+---+------+----------+---------------+-----+-----+-----+-----------+-------------+------+------+-------------+-----------+------------+----------+-------+-------+
| id|season|match_type|player_of_match|venue|team1|team2|toss_winner|toss_decision|winner|result|result_margin|target_runs|target_overs|super_over|umpire1|umpire2|
+---+------+----------+---------------+-----+-----+-----+-----------+-------------+------+------+-------------+-----------+------------+----------+-------+-------+
+---+------+----------+---------------+-----+-----+-----+-----------+-------------+------+------+-------------+-----------+------------+----------+-------+-------+



In [62]:
df_match_cleaned[df_match_cleaned['match_type']=="Semi Final"].show()

+------+------+----------+---------------+--------------------+--------------------+-------------------+--------------------+-------------+--------------------+-------+-------------+-----------+------------+----------+-----------+-----------+
|    id|season|match_type|player_of_match|               venue|               team1|              team2|         toss_winner|toss_decision|              winner| result|result_margin|target_runs|target_overs|super_over|    umpire1|    umpire2|
+------+------+----------+---------------+--------------------+--------------------+-------------------+--------------------+-------------+--------------------+-------+-------------+-----------+------------+----------+-----------+-----------+
|336038|  2008|Semi Final|      SR Watson|    Wankhede Stadium|      Delhi Capitals|   Rajasthan Royals|      Delhi Capitals|        field|    Rajasthan Royals|   runs|          105|        193|          20|         N|  BF Bowden|RE Koertzen|
|336039|  2008|Semi Final|  

As from 2008 to 2010 the qualifiers were called the semi final so we have to maually degrade the semifals into qf 1 and qf2 

In [65]:
import shutil

# Define file path
output_folder = "matches_cleaned"

# Save DataFrame to CSV in a folder
df_match_nw.coalesce(1).write.mode("overwrite").option("header", "true").csv(output_folder)

# Find the generated CSV file inside the folder
import os
csv_file = [f for f in os.listdir(output_folder) if f.endswith(".csv")][0]

# Rename and move it outside the folder
shutil.move(f"{output_folder}/{csv_file}", "matches_cleaned.csv")

# Remove the empty folder
shutil.rmtree(output_folder)